In [1]:
! pip3 install --upgrade --quiet --user google-cloud-aiplatform==1.88.0

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 2.14.2 requires protobuf<7.0,==6.31.1, but you have protobuf 5.29.5 which is incompatible.
kfp-pipeline-spec 2.14.0 requires protobuf<7.0,==6.31.1, but you have protobuf 5.29.5 which is incompatible.


In [2]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)

qwiklabs-gcp-03-b2ab7d4155ac


In [2]:
import requests
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerationConfig,
    GenerativeModel,
    Part,
    Tool,
)

In [35]:
#from google.generativeai.tool import Tool, FunctionDeclaration

# Define the Python functions first
def add(num1, num2):
  """Adds two numbers, prints the result, and returns it."""
  total = num1 + num2
  print("Calling add function")
  return total

def multiply(num1, num2):
  """Multiplies two numbers, prints the result, and returns it."""
  product = num1 * num2
  print("Calling multiply function")
  return product

# Define the math Tool for the model
math_tool = Tool(
    function_declarations=[
        FunctionDeclaration(
            name="add",
            description="Adds two numbers.",
            parameters={
                "type": "object",
                "properties": {
                    "num1": {
                        "type": "number",
                        "description": "The first number."
                    },
                    "num2": {
                        "type": "number",
                        "description": "The second number."
                    }
                },
                "required": ["num1", "num2"]
            }
        ),
        FunctionDeclaration(
            name="multiply",
            description="Multiplies two numbers.",
            parameters={
                "type": "object",
                "properties": {
                    "num1": {
                        "type": "number",
                        "description": "The first number."
                    },
                    "num2": {
                        "type": "number",
                        "description": "The second number."
                    }
                },
                "required": ["num1", "num2"]
            }
        )
    ]
)

In [36]:
#import google.generativeai as genai

# NOTE: This assumes the 'math_tool' from the previous step is already defined.

# Define the system instructions for the model
system_instructions = """
You are a helpful assistant. Fulfill the user's instructions, including telling jokes.
When asked to add or multiply numbers, you must call the provided functions.
You may call one function after the other if needed.
After calling a function, repeat the final numerical result to the user.
"""

# Initialize the Generative Model
model = GenerativeModel(
    model_name="gemini-2.0-flash-001",
    tools=[math_tool],
    generation_config={"temperature": 0},
    system_instruction=system_instructions
)

In [37]:
chat = model.start_chat()

In [38]:
available_functions = {
    "add": add,
    "multiply": multiply,
}
def handle_response(response):
    """Handles the model's response, calling functions if requested."""

    # If there is a function call, extract the first one.
    # Otherwise, print the model's text response and stop.
    if response.candidates[0].function_calls:
        function_call = response.candidates[0].function_calls[0]
    else:
        print(response.text)
        return

    # Check if the requested function is one we have defined
    if function_call.name in available_functions:
        # Select the correct function from our dictionary
        function_to_call = available_functions[function_call.name]
        
        # Extract the arguments from the model's request
        args = {key: value for key, value in function_call.args.items()}
        
        # Call the local Python function with the provided arguments
        function_response = function_to_call(**args)
        
        # Send the function's result back to the model so it can formulate a text response
        api_response = chat.send_message(
            Part.from_function_response(
                name=function_call.name,
                response={"result": function_response},
            ),
        )
        
        # Make a recursive call to handle the model's next response
        handle_response(api_response)

    else:
        # This would happen if the model tries to call a function you haven't defined
        print(f"Error: Model tried to call unknown function '{function_call.name}'")

In [39]:
response = chat.send_message("Tell me a joke?")
handle_response(response)

Why don't scientists trust atoms?

Because they make up everything!



In [40]:
response = chat.send_message("I have 7 pizzas each with 16 slices. How many slices do I have?")
handle_response(response)

Calling multiply function
You have 112 slices.



In [41]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. How many pizzas did they bring together?")
handle_response(response)

Calling add function
They brought 7 pizzas together.



In [32]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. There are 16 slices per pizza. How many slices are there?")
handle_response(response)    

The sum of 3 and 4 is 7.
The product of 7 and 16 is 112.
There are 112 slices in total.



In [48]:
response = chat.send_message("Doug brought 3 pizzas. Andrew brought 4 pizzas. There are 16 slices per pizza. How many slices are there?")
handle_response(response)

Calling add function
Calling multiply function
There are 112 slices in total.

